## Problem Statement
The objective of this project is to predict the location and type of defects found in steel manufacturing using the images provided. The images are named with a unique ImageId, and our task is to segment each image and classify the defects in the test set

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import random
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
from PIL import Image 
import tensorflow as tf
#import tensorflow
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import UpSampling2D, Conv2D, Activation, LeakyReLU, BatchNormalization,Input,Conv2DTranspose
from tensorflow. keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler,Callback
from tqdm import tqdm_notebook
#from .layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

In [19]:
train = pd.read_csv('C:/Users/sonali/Desktop/DS/Semester_4/DLproject/severstal-steel-defect-detection/train.csv')
train.shape

(7095, 3)

In [20]:
def defeat_position(img_id, class_id):
    
    train_s = train[train.ImageId_ClassId==f"{img_id}_{class_id}"]
    #print(train_s.EncodedPixels.values[0])
    
    if not isinstance(train_s.EncodedPixels.values[0], str) and np.isnan(train_s.EncodedPixels.values[0]):
        return np.array([]), np.array([])
    
    encoded_pixels = [int(i) for i in train_s.EncodedPixels.values[0].split(" ")]

    pixcels = []
    for pos, offset in zip(encoded_pixels[0:len(encoded_pixels):2], encoded_pixels[1:len(encoded_pixels):2]):
        pixcels.extend(list(range(pos, pos+offset)))
    pixcels = np.array(pixcels)    
    x = pixcels // 256
    y = pixcels % 256  
    return x, y

def show_segmented_image(img_file, x, y, class_id):
    im = np.array(Image.open(img_file))
    
    if class_id == 4:
        im[y, x, 1] += 50
        im[y, x, 2] += 50
    else:
        im[y, x, class_id-1] += 50
        
    im = np.clip(im, 0, 255)

    plt.figure(figsize=(25,5))
    plt.imshow(im)
    plt.xticks([]);plt.yticks([]);
    plt.show()
    
def visualize_defect(class_id, n_show=20):
    cnt = 0
    for img_file in train_files:
        img_id = img_file.split('/')[-1]
        x, y = defeat_position(img_id, class_id)
        if len(x)==0:
            continue
        print(img_file, class_id)
        show_segmented_image(img_file, x, y, class_id)
        cnt += 1
        if cnt > n_show: break

In [21]:
from glob import glob
# set paths to train and test image datasets
train_files = 'C:/Users/sonali/Desktop/DS/Semester_4/DLproject/severstal-steel-defect-detection/train_images/'
test_files = 'C:/Users/sonali/Desktop/DS/Semester_4/DLproject/severstal-steel-defect-detection/test_images/'

# load dataframe with train labels
train_fns = sorted(glob(train_files + '*.jpg'))
test_fns = sorted(glob(test_files + '*.jpg'))

print('There are {} images in the train set.'.format(len(train_fns)))
print('There are {} images in the test set.'.format(len(test_fns)))

There are 12568 images in the train set.
There are 5506 images in the test set.


In [22]:
na_cut = train.EncodedPixels.isna()
count_na = (~na_cut).astype(int)

In [23]:
# The number of defect
count_na.value_counts()

1    7095
Name: EncodedPixels, dtype: int64

In [13]:
# The number of each defect class
#pd.Series(train.loc[~na_cut].ImageId_ClassId.str.split("_", expand=True).values[:, 1]).value_counts().sort_index()

In [71]:
import numpy as np
def rle2mask(rle):
  # If rle is empty or null
  if(len(rle)<1):
    return np.zeros((128,800) ,dtype=np.uint8)
    height = 256
    width = 1600
  
  # Defining the length of mask. This will be 1d array and later will be reshaped to 2d.
    mask = np.zeros(height*width ).astype(np.uint8)
  # We will have an array that wil contain rle
    array = np.asarray([int(x) for x in rle.split()])
    start = array[0::2]-1 # this willl contain the start of run length
    length = array[1::2] # this will contain the length of each rle.
  
    
    
  
    return mask.reshape( (height,width), order='F' )[::2,::2]

In [72]:
def mask2rle(img):
        '''
        img: numpy array, 1 - mask, 0 - background
        Returns run length as string formated
        '''
        #print(img.shape)
        pixels= img.T.flatten()
        pixels = np.concatenate([[0], pixels, [0]])
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        return ' '.join(str(x) for x in runs)

In [52]:
train_df = train.iloc[:int(0.80*len(train))]
test_df = train.iloc[int(0.80*len(train)):int(0.9*len(train))]
cv_df = train.iloc[int(0.9*len(train)):]

In [53]:
print(train_df.shape)
print(test_df.shape)
print(cv_df.shape)

(5676, 3)
(709, 3)
(710, 3)


In [58]:
import keras
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.data_utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, df, batch_size = 16, subset="train", shuffle=False, 
                 preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        
        if self.subset == "train":
            self.data_path = 'C:/Users/sonali/Desktop/DS/Semester_4/DLproject/severstal-steel-defect-detection/train_images/'
        elif self.subset == "test":
            self.data_path = 'C:/Users/sonali/Desktop/DS/Semester_4/DLproject/severstal-steel-defect-detection/train_images/'
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    #fliping the images horizontally and samplewise std normalization
    def __getitem__(self, index):
        train_datagen = ImageDataGenerator()
        param = {'flip_horizontal':True, 'samplewise_std_normalization' : True}
        
        X = np.empty((self.batch_size,128,800,3),dtype=np.float32)
        y = np.empty((self.batch_size,128,800,4),dtype=np.int8)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            img = Image.open(self.data_path + f).resize((800,128))
            X[i,] = train_datagen.apply_transform(x = img, transform_parameters = param)
            if self.subset == 'train': 
                #run-length encoding on the pixel values
                for j in range(4):
                    mask = rle2mask(self.df['e'+str(j+1)].iloc[indexes[i]])
                    y[i,:,:,j] = train_datagen.apply_transform(x = mask, transform_parameters = param)
        if self.preprocess!=None: X = self.preprocess(X)
        if self.subset == 'train': return X, y
        else: return X

In [59]:
from keras import backend as K
from keras.losses import binary_crossentropy
# Competetion Metric
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
  
def  bce_dice_loss(y_true, y_predict):
    return binary_crossentropy(y_true, y_predict) + (1-dice_coef(y_true, y_predict))

In [60]:
import tensorflow as tf
inputs = Input((256,1600,3)) # 256
down = Conv2D(32, (3, 3), padding='same')(inputs)
print(down)
down = BatchNormalization()(down)
down = Activation('relu')(down)
down_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(down) #128
center = Conv2D(128, (3, 3), padding='same')(down_pool)
center = BatchNormalization()(center)
center = Activation('relu')(center) # center
up = UpSampling2D((2, 2))(center)
up = tf.keras.layers.concatenate([down, up], axis=3)
up = Conv2D(32, (3, 3), padding='same')(up)
up = BatchNormalization()(up)
up = Activation('relu')(up) # 256
classify = Conv2D(4, (1, 1), activation='sigmoid')(up)
model = Model(inputs=inputs, outputs=classify)
model.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy',dice_coef])

Tensor("conv2d_20/Identity:0", shape=(None, 256, 1600, 32), dtype=float32)


In [73]:

 
train_batches = DataGenerator(train_df,shuffle=True)
valid_batches = DataGenerator(cv_df)
history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = 5, verbose=1)

Epoch 1/5


KeyError: 'e1'